# purpose

- record steps and missteps in setting up dj for lab

# imports

In [3]:
import datajoint as dj
import glob, os, sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'datajoint'

# procedure

In [2]:
dj.config['database.host'] = 'vm7146.int.janelia.org'
dj.config['database.password'] = 'nq7^rGqt$jWuJm2M'
dj.config['database.user'] = 'djAdmin'
dj.conn(use_TLS=False)

NameError: name 'dj' is not defined

In [7]:
#debug by testing connection - requires manual stop
!ping vm7146.int.janelia.org

PING vm7146.int.janelia.org (10.37.7.146) 56(84) bytes of data.
64 bytes from vm7146.int.janelia.org (10.37.7.146): icmp_seq=1 ttl=63 time=0.600 ms
64 bytes from vm7146.int.janelia.org (10.37.7.146): icmp_seq=2 ttl=63 time=0.261 ms
64 bytes from vm7146.int.janelia.org (10.37.7.146): icmp_seq=3 ttl=63 time=0.349 ms
64 bytes from vm7146.int.janelia.org (10.37.7.146): icmp_seq=4 ttl=63 time=0.211 ms
^C

--- vm7146.int.janelia.org ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3056ms
rtt min/avg/max/mdev = 0.211/0.355/0.600/0.149 ms


In [12]:
# connecting to the vm is possible, so trying an alt dj.conn call
dj.config['database.host'] = 'vm7146.int.janelia.org'
dj.config['database.password'] = 'nq7^rGqt$jWuJm2M'
dj.config['database.user'] = 'djAdmin'
dj.conn(use_tls=False)

DataJoint connection (connected) djAdmin@vm7146.int.janelia.org:3306

In [13]:
#using tutorial from tutorials.datajoint.io/beginner/building-first-pipeline/python/first-table.html, which only opens in microsoft edge browser for me 2022/08/10
schema = dj.schema('ejd_tutorial',locals())
# adding the locals() allows us to see all other tables created and call them simply by their name

In [14]:
#ok let's use the schema object to create a new table, mouse
# we "decorate" the class Mouse with the schema object, meaning that Mouse table is specified inside the schema object that points to tutorial.
# we use dj.Manual because we're going to manually enter this info
# the # mouse line gives us space for a reader-friendly comment about what the table is/contains
# the rest is formatted as attribute: datatype # comment
# the --- separates primary keys (UNIQUE identifiers) and secondary keys (can be repeated)
@schema
class Mouse(dj.Manual):
    definition = """
    # mouse
    mouse_id: int    #unique mouse id, an integer
    ---
    dob: date   #mouse date of birth, a date 
    sex_gonadal: enum('M','F','I','U') #gonadal sex, male, female, intersex, unclassified or unknown
    """

In [16]:
mouse = Mouse()
mouse.insert1( (0,'2022-08-10','F'))

In [17]:
mouse

"mouse_id unique mouse id, an integer","dob mouse date of birth, a date","sex_gonadal gonadal sex, male, female, intersex, unclassified or unknown"
0,2022-08-10,F


In [20]:
data_dict = {
    'mouse_id': 1,
    'dob': '2022-08-10',
    'sex_gonadal':'U'
}

In [21]:
mouse.insert1(data_dict)

In [22]:
mouse

"mouse_id unique mouse id, an integer","dob mouse date of birth, a date","sex_gonadal gonadal sex, male, female, intersex, unclassified or unknown"
0,2022-08-10,F
1,2022-08-10,U


In [23]:
data_dict_2 =  {
    'mouse_id': 2,
    'dob': '2022-08-10',
    'sex_gonadal':'M'
}
data_dict_3 =  {
    'mouse_id': 3,
    'dob': '2022-08-10',
    'sex_gonadal':'I'
}
mouse.insert([data_dict_2,data_dict_3])
mouse

"mouse_id unique mouse id, an integer","dob mouse date of birth, a date","sex_gonadal gonadal sex, male, female, intersex, unclassified or unknown"
0,2022-08-10,F
1,2022-08-10,U
2,2022-08-10,M
3,2022-08-10,I


# Borrowed from Brody and IBL

In [27]:
# add graphviz to path, taken from https://github.com/Brody-Lab/bl_pipeline_python/blob/main/notebooks/00-Datajoint-configuration.ipynb
if os.name == 'nt':
    if 'C:\\Program Files\\Graphviz\\bin' not in os.environ["PATH"]:
        os.environ["PATH"] += ';C:\\Program Files\\Graphviz\\bin'

In [28]:
# prefixes are useful - like dennis_


if 'custom' not in dj.config:
    dj.config['custom'] = dict()
dj.config['custom']['database.prefix'] = 'bl_new_'
dj.config.save_local() #should make dj_local_conf.json

In [32]:
# later we can make this into a standalone file in a repo, and import it

import os
import pathlib

def chdir_to_root():

    root_dir_found = 0
    conf_file_found = 0
    while 1:
        current_dir = os.getcwd()
        u19_dir = pathlib.Path(current_dir,'bl_pipeline')
        if os.path.isdir(u19_dir):
            root_dir_found = 1
            if os.path.isfile(pathlib.Path(current_dir,'dj_local_conf.json')):
                conf_file_found = 1
        if root_dir_found:
            break
        os.chdir('..')
        new_current_dir = os.getcwd()
        if str(current_dir) == str(new_current_dir):
            break

    return root_dir_found, conf_file_found


def try_find_conf_file():

    root_dir_found, conf_file_found = chdir_to_root()
    if root_dir_found and conf_file_found:
        print('Local configuration file found !!, no need to run the configuration (unless configuration has changed)')
    elif root_dir_found:
        print('Local configuration file not found. Ignore this if you have a global config. Run configuration notebook otherwise')
    else:
        print('Root dir not found, change this notebook to the project folder')

### useful bits but not particularly useful here during setup

from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()
import getpass
dj.config['database.password'] = getpass.getpass() # enter the password securely

https://github.com/Brody-Lab/bl_pipeline_python/blob/main/notebooks/tutorials/Explore_Sessions_Data.ipynb

# return to procedure

In [55]:
# defining a table module
# this is making a .py file that is read as a module in python so we can import our tables


for_file = ["import datajoint as dj",
            "schema = dj.schema('tutorial',locals())",    
            "@schema","class Mouse(dj.Manual):",
            "    definition = \"\"\"",
            "    # mouse",
            "    mouse_id: int    #unique mouse id, an integer",
            "    ---",
            "    dob: date   #mouse date of birth, a date", 
            "    sex_gonadal: enum('M','F','I','U') #gonadal sex, male, female, intersex, unclassified or unknown\"\"\""]


def make_table_def_as_file(file_loc,table_data):
    """ takes a location and a list of lines to write to a table .py file"""
    if not os.path.exists(os.path.dirname(file_loc)):
        print("WARNING invalid path, {} does not exist".format(os.path.dirname(file_loc)))
    if "py" not in file_loc:
        print("adding .py to file_loc")
        file_loc = "{}.py".format(file_loc)
    print("saving table at {}".format(file_loc))
    with open(file_loc,"w") as f:
        for line in table_data:
            f.write("{}\n".format(line))
        f.close()
        
file_loc = 'tutorial_tables.py'
make_table_def_as_file(file_loc,for_file)

WARNING invalid path,  does not exist
saving table at tutorial_tables.py


In [56]:
from tutorial_tables import *

In [57]:
# fetch some data
data = mouse.fetch()
data

array([(0, datetime.date(2022, 8, 10), 'F'),
       (1, datetime.date(2022, 8, 10), 'U'),
       (2, datetime.date(2022, 8, 10), 'M'),
       (3, datetime.date(2022, 8, 10), 'I')],
      dtype=[('mouse_id', '<i8'), ('dob', 'O'), ('sex_gonadal', 'O')])

In [58]:
# fetch some data as a dict
data = mouse.fetch(as_dict=True)
data

[{'mouse_id': 0, 'dob': datetime.date(2022, 8, 10), 'sex_gonadal': 'F'},
 {'mouse_id': 1, 'dob': datetime.date(2022, 8, 10), 'sex_gonadal': 'U'},
 {'mouse_id': 2, 'dob': datetime.date(2022, 8, 10), 'sex_gonadal': 'M'},
 {'mouse_id': 3, 'dob': datetime.date(2022, 8, 10), 'sex_gonadal': 'I'}]

In [62]:
# another way to fetch data
sex, ids = mouse.fetch('sex_gonadal','mouse_id')
print(sex, ids)

mouse & 'sex_gonadal = "M"'

['F' 'U' 'M' 'I'] [0 1 2 3]


"mouse_id unique mouse id, an integer","dob mouse date of birth, a date","sex_gonadal gonadal sex, male, female, intersex, unclassified or unknown"
2,2022-08-10,M


## add users

- https://docs.datajoint.org/python/admin/3-accounts.html

- using common prefixes are helpful for this step! e.g.
    - labwide_
    - ejd_
    - stonyford_
    - el-hady_
    - voigts_
    - egnor_
    - skokholm_

- need to do this on the mysql side!
    - `CREATE USER 'ejd'@'%' IDENTIFIED BY 'password';`
    - to see current users: 
        `SELECT user, host from mysql.server`
    - to change privileges
```
GRANT SELECT, INSERT ON `common\_%`.* TO 'ejd'@'%';
GRANT ALL PRIVILEGES ON `pipeline\_%`.* TO 'ejd'@'%';
GRANT ALL PRIVILEGES ON `alice\_%`.* TO 'ejd'@'%';
GRANT ALL PRIVILEGES ON `user_ejd\_%`.* TO 'ejd'@'%';

```

## use elements
https://www.youtube.com/watch?v=NRqpKNoHEY0 at 4:57
want to have a workflow_session folder, with files including

```
__init__.py
export.py
ingest.py
paths.py
workflow.py
version.py
```

and in workflow.py, it imports the multiple schemas using 

```
import datajoint as dj
from element_lab import lab

if 'custom' not in dj.config:
    dj.config['custom'] = {}
    
db_prefix = dj.config['custom']..get('database.prefix','')

lab.activate(db_prefix + 'lab')
```

In [68]:
# import elements
from element_lab import lab

In [73]:
lab.activate('schema')
lab.Lab()

lab Abbreviated lab name,lab_name full lab name,institution,address,time_zone 'UTC±X' format for NWB export


# our data structure

# notes

### types: 
https://docs.datajoint.org/python/definition/06-Datatypes.html
1. tinyint
2. tinyint unsigned
3. smallint
4. smallint unsigned
5. int
6. int unsigned
7. enum
    - e.g. enum("isoflurane", "urethane")
8. date
    - e.g. YYYY-MM-DD
9. time
    - HH:MM:SS
10. datetime
    - YYYY-MM-DD HH:MM:SS
11. timestamp
    - same as datetime but adjusted to local time zone
12. char(N)
    - always takes N bytes to store, only characters up to N
13. varchar(N)
    - saves variable length up to N, uses len(str)+1 or len(str)+2 bytes of storage
14. float
15. double
16. decimal (N,F)
    - decimal with N total digits and F fractional digits. lighter on storage and recall than float, good for things like money in dollars, cents
17. mediumblob
    - arbitrary numeric array up to 16 MiB
18. blob
    - same, up to 65 KiB
19. tinyblob
    - same, up to 256 bytes - header

---

#### datajoint-only types
    - https://docs.datajoint.org/python/definition/06.5-External-Data.html

20. attach
    - file attachment, allowing send/receive of an opaque data file to/from a workflow
    - For DataJoint attach attributes, DataJoint will copy the input file into a DataJoint store, hash the file contents, and track the input file name. Subsequent fetch operations will transfer a copy of the file to the local directory of the Python process and return a pointer to it’s location for subsequent client usage. This allows arbitrary files to be uploaded or attached to a DataJoint schema for later use in processing. File integrity is preserved by checksum comparison against the attachment data and verifying the contents during retrieval.
21. filepath@store
    - filepath to link non-DJ managed files into the workflow
    - To define a table using the filepath datatype, an existing DataJoint store should be created and then referenced in the new table definition. For example, given a simple store:


### external store(age)
- https://docs.datajoint.org/python/admin/5-blob-config.html#external
- longblob can contain up to 4Gb data after compression, but this slows down query-ing tables
- good rule of thumb: > 10Mb should be stored elsewhere

- defining an externally-stored attribute: blob@storename, same as longblob but with external storage, not in the database itself

``` # Motion aligned movies
-> twophoton.Scan
---
aligned_movie :  blob@external  # motion-aligned movie in 'external' store
```

- each store has a specification in connection config, e.g.
```
dj.config['stores'] = {
  'external': dict(  # 'regular' external storage for this pipeline
                protocol='s3',
                endpoint='s3.amazonaws.com:9000',
                bucket = 'testbucket',
                location = 'datajoint-projects/lab1',
                access_key='1234567',
                secret_key='foaf1234'),
  'external-raw': dict( # 'raw' storage for this pipeline
                protocol='file',
                location='/net/djblobs/myschema')
}
# external object cache - see fetch operation below for details.
dj.config['cache'] = '/net/djcache'
```


### lengthy example of setting up external storage, Emily TODO
```
The following steps must be performed to enable external storage:

1. Assign external location settings for each storage as shown in the Step 1 example above.

2. Use dj.config for configuration.

3. protocol [s3, file] Specifies whether s3 or file external storage is desired.

4. endpoint [s3] Specifies the remote endpoint to the external data for all schemas as well as the target port.

5. bucket [s3] Specifies the appropriate s3 bucket organization.

6. location [s3, file] Specifies the subdirectory within the root or bucket of store to preserve data. External objects are thus stored remotely with the following path structure: <bucket (if applicable)>/<location>/<schema_name>/<subfolding_strategy>/<object>.

7. access_key [s3] Specifies the access key credentials for accessing the external location.

8. secret_key [s3] Specifies the secret key credentials for accessing the external location.

9. secure [s3] Optional specification to establish secure external storage connection with TLS (aka SSL, HTTPS). Defaults to False.

10. Optionally, for each schema specify the cache folder for local fetch cache.

11. This is done by saving the path in the cache key of the DataJoint configuration dictionary:

12. dj.config['cache'] = '/temp/dj-cache'
```